In [1]:
from ae_feature_extraction import *

# feature_list_expend = encode_feature_extraction(train_data_expend)  # 获得切片后测试集的featurelist
# feature_list_test = encode_feature_extraction(test_dataset)  # 获得testdata的featurelist



In [3]:
def encode_feature_extraction1(instance):
    """
    instance is a 类的实例，含有原有的时间序列数据
    return the extracted feature as a list n*num_feature
    """
    aemodel_encode = LSTMAutoEncoder()
    aemodel_encode.load_state_dict(torch.load("./model_checkpoints_ae/ae/model_ae.pt", map_location="cpu"))
    all_features_list = []
    for idx in range(len(instance)):
        with torch.no_grad():
            ae_result = aemodel_encode.encoder1(instance[idx]["input"].unsqueeze(0))[0]#.squeeze(0)
            ae_result=aemodel_encode.encoder2(ae_result)[1][0].squeeze()
        all_features_list.append(ae_result)
    return all_features_list

In [4]:
feature_list_expend1 = encode_feature_extraction1(train_dataset_expend)  # 获得切片后测试集的featurelist_raw
feature_list_test1 = encode_feature_extraction1(test_dataset)  # 获得testdata的featurelist_raw

In [5]:
train_dataset_expend_encoded = AircraftDataset_expend_feature_extraction(df_train, train_label, feature_list_expend1)
train_encoded_loader = DataLoader(
    train_dataset_expend_encoded,
    batch_size=CFG.batch_size,
    shuffle=True,
    drop_last=True,
)
test_dataset = AircraftDataset_no_expend_feature_extraction(df_train, test_label, feature_list_test1)
test_encoded_loader = DataLoader(
    test_dataset,
    batch_size=1,
    shuffle=False)
# classifier_in = KMeans(n_clusters=CFG.num_in_feature_classes, random_state=CFG.seed).fit(get_input(test_dataset))
model = CustomModel(CFG.ae_hidden_layer*2).to(device)
loss_function = CustomLoss()

In [6]:
def fit_his_unit_in_unit_0(train_loader, test_loader_batch, in_unit_index, folder, RESUME=False):
    """
    传入 in_unit 的feature来获得kernel完成预测
    :param train_loader: 含有多个batch, 是字典形式的
    :param test_loader_batch: test loader 的一个batch， 是字典形式的
    :param in_unit_index: 记录in unit的index
    :return: None
    """
    # last_loss = torch.tensor([1e5], dtype=torch.float).to(CFG.device)
    optimizer = getattr(torch.optim, CFG.optimizer)(model.parameters(), lr=CFG.lr)  # 优化器
    scheduler = getattr(torch.optim.lr_scheduler, CFG.scheduler)(optimizer, gamma=CFG.sc_Gamma)  # 指数型学习率
    start_epoch = -1
    for layer in model.modules():
        if isinstance(layer, nn.Linear):
            nn.init.xavier_uniform_(layer.weight.data, gain=nn.init.calculate_gain('relu'))

    if RESUME:
        path_checkpoint = folder + '/ckpt_unit_%s.pth' % (str(in_unit_index))  # 断点路径
        if os.path.isfile(path_checkpoint):
            checkpoint = torch.load(path_checkpoint, map_location=CFG.device)  # 加载断点

            model.load_state_dict(checkpoint['net'])  # 加载模型可学习参数

            optimizer.load_state_dict(checkpoint['optimizer'])  # 加载优化器参数
            start_epoch = checkpoint['epoch']  # 设置开始的epoch
            scheduler.load_state_dict(checkpoint['scheduler'])
        else:
            None

    for epoch in range(start_epoch + 1, CFG.epoches):
        for data in train_loader:
            pred_batch = model(data["input"].to(device))
            target_batch = data["RUL"].unsqueeze(-1).to(device)
            train_data = data["input"].to(device)
            test_data = test_loader_batch["input"].to(device)
            loss = loss_function(pred_batch, target_batch, train_data, test_data)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            scheduler.step()
        # scheduler.step(loss,last_loss)
        # last_loss = loss
        if CFG.print_training_process and epoch % 10 == 0:
            print(f"epoch:{epoch}, loss:{loss.item()},lr:{optimizer.state_dict()['param_groups'][0]['lr']}")
            checkpoint = {
                "net": model.state_dict(),
                'optimizer': optimizer.state_dict(),
                "epoch": epoch,
                'scheduler': scheduler.state_dict()
            }
            if not os.path.isdir(folder):
                os.mkdir(folder)
            torch.save(checkpoint, folder + '/ckpt_unit_%s.pth' % (str(in_unit_index)))

        if loss.mean() < CFG.jump_out_value:
            save_model_weights(model, f"model_in_unit_{in_unit_index}.pt",
                               cp_folder=folder)
            break
        elif epoch == (CFG.epoches - 1):
            print(f"epoch:{epoch}, loss:{loss.item()}")
            save_model_weights(model, f"model_in_unit_{in_unit_index}.pt",
                               cp_folder=folder)

In [7]:
CFG.sc_Gamma=0.999

In [ ]:
for idx, test_data in enumerate(test_encoded_loader):
    fit_his_unit_in_unit_0(train_encoded_loader, test_data, idx,
                                   "./model_checkpoints_ae/kernel without classify", RESUME=False)

epoch:0, loss:12200.6162109375,lr:0.09890548353295386
epoch:10, loss:3649.643798828125,lr:0.08859803203984777
epoch:20, loss:3174.1259765625,lr:0.07936477332643059
epoch:30, loss:3444.789306640625,lr:0.07109376021267357
epoch:40, loss:3537.2412109375,lr:0.06368471211262068
epoch:50, loss:3310.347900390625,lr:0.05704779919833771
epoch:60, loss:3354.708740234375,lr:0.05110255327241894
epoch:70, loss:3263.291259765625,lr:0.04577689214409707
epoch:80, loss:3327.454345703125,lr:0.041006245680160425
epoch:90, loss:3292.71630859375,lr:0.03673277293461932
epoch:100, loss:3095.048583984375,lr:0.032904660865335375
epoch:110, loss:3598.960693359375,lr:0.029475496135014385
epoch:120, loss:3159.05078125,lr:0.026403702380063794
epoch:130, loss:3298.459716796875,lr:0.023652036124570106
epoch:140, loss:3336.44384765625,lr:0.021187135227685356
epoch:150, loss:3198.087646484375,lr:0.018979114389644735
epoch:160, loss:3153.134765625,lr:0.017001202812192157
epoch:170, loss:3422.08837890625,lr:0.0152294196